In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, models, layers, optimizers, metrics
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = "TRUE"

In [2]:
b_size = 20

train_datagen = ImageDataGenerator(rescale=1./255,
                                  horizontal_flip=True,     #수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,  #전체 크기의 10% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1, #마찬가지로 위, 아래로 이동합니다.
                                  #rotation_range=5,
                                  #shear_range=0.7,
                                  #zoom_range=[0.9, 2.2],
                                  #vertical_flip=True,
                                  fill_mode='nearest') #주위의 px 값으로 채움

In [3]:
train_generator = train_datagen.flow_from_directory(
       'data/dogs_vs_cats/train',   #학습셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=b_size,
       class_mode='binary')

#테스트 셋은 이미지 부풀리기 과정을 진행하지 않습니다.
test_datagen = ImageDataGenerator(rescale=1./255)  

test_generator = test_datagen.flow_from_directory(
       'data/dogs_vs_cats/test',   #테스트셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=b_size,
       class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
transfer_model.trainable = False
transfer_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [5]:
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dense(1, activation='sigmoid'))
finetune_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 64)                524352    
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 15,239,105
Trainable params: 524,417
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
finetune_model.compile(loss='binary_crossentropy', 
                       optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
model_path = './dogs-vs-cats.hdf5'
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)

In [8]:
history = finetune_model.fit(
      train_generator,
      epochs=50,
      validation_data=test_generator,
      callbacks=[early_stopping, checkpoint])

Epoch 1/50
1000/1000 [==============================] - ETA: 0s - loss: 0.2853 - accuracy: 0.8773
Epoch 1: val_loss improved from inf to 0.22373, saving model to .\dogs-vs-cats.hdf5
1000/1000 [==============================] - 1251s 1s/step - loss: 0.2853 - accuracy: 0.8773 - val_loss: 0.2237 - val_accuracy: 0.9062
Epoch 2/50
1000/1000 [==============================] - ETA: 0s - loss: 0.2425 - accuracy: 0.8953
Epoch 2: val_loss did not improve from 0.22373
1000/1000 [==============================] - 1180s 1s/step - loss: 0.2425 - accuracy: 0.8953 - val_loss: 0.2278 - val_accuracy: 0.9068
Epoch 3/50
1000/1000 [==============================] - ETA: 0s - loss: 0.2237 - accuracy: 0.9058
Epoch 3: val_loss did not improve from 0.22373
1000/1000 [==============================] - 1183s 1s/step - loss: 0.2237 - accuracy: 0.9058 - val_loss: 0.2305 - val_accuracy: 0.9068
Epoch 4/50
1000/1000 [==============================] - ETA: 0s - loss: 0.2120 - accuracy: 0.9094
Epoch 4: val_loss improve